In [10]:
import pandas as pd
import numpy as np

import submission_util as subutil
import submission_config as subconfig

df1 = pd.read_csv(subconfig.OUTPUT_PATH / 'alpha_feat_insoo_lagged_ReverseSharpe-final.csv', index_col = 0)
df2 = pd.read_csv(subconfig.OUTPUT_PATH / '제출_파일2(SHARPE).csv', index_col = 0)
df3 = pd.read_csv(subconfig.OUTPUT_PATH / 'alpha_feat_insoo_lagged_ReverseSharpe_disparity_added_2.csv', index_col = 0)

In [11]:
df1.sort_index(inplace = True)
df2.sort_index(inplace = True)
df3.sort_index(inplace = True)
df1['Long Voting'] = df1['순위'].apply(lambda x: 1 if x <= 200 else 0)
df2['Long Voting'] = df2['순위'].apply(lambda x: 1 if x <= 200 else 0)
df3['Long Voting'] = df3['순위'].apply(lambda x: 1 if x <= 200 else 0)
df1['Short Voting'] = df1['순위'].apply(lambda x: 1 if x > 1800 else 0)
df2['Short Voting'] = df2['순위'].apply(lambda x: 1 if x > 1800 else 0)
df3['Short Voting'] = df3['순위'].apply(lambda x: 1 if x > 1800 else 0)


In [12]:
final = pd.DataFrame()

final['Long Voting'] = df1['Long Voting'] + df2['Long Voting'] + df3['Long Voting']
final['Short Voting'] = df1['Short Voting'] + df2['Short Voting'] + df3['Short Voting']
final['Average Rank'] = round((df1['순위'] + df2['순위'] + df3['순위'])/3, 2)

In [13]:
final['Final Decision'] = 0  # Fill NaN with 0
final['순위'] = 0
rank_counter = 1

for vote in [3, 2, 1]:
    subset = final[final['Long Voting'] == vote]
    subset = subset.nsmallest(200, 'Average Rank')  # Only select the top 200
    final.loc[subset.index, 'Final Decision'] = 1  # Update the 'Final Decision' in the original DataFrame

    # Sort the subset based on 'Average Rank'
    subset_sorted = subset.sort_values('Average Rank')
    
    # Assign ranks starting from the current rank_counter
    final.loc[subset_sorted.index, '순위'] = range(rank_counter, rank_counter + len(subset_sorted))
    
    # Update the rank_counter
    rank_counter += len(subset_sorted)

# Reset rank_counter for 'Short Voting'
rank_counter = 1801

for vote in [3, 2, 1]:
    subset = final[final['Short Voting'] == vote]
    subset = subset.nsmallest(200, 'Average Rank')  # Only select the top 200
    final.loc[subset.index, 'Final Decision'] = 1  # Update the 'Final Decision' in the original DataFrame

    # Sort the subset based on 'Average Rank'
    subset_sorted = subset.sort_values('Average Rank')
    
    # Assign ranks starting from the current rank_counter
    final.loc[subset_sorted.index, '순위'] = range(rank_counter, rank_counter + len(subset_sorted))
    
    # Update the rank_counter
    rank_counter += len(subset_sorted)
 

In [14]:
# Identify rows where 'Final Decision' is still 0
remaining_subset = final[final['Final Decision'] == 0]

# Sort this subset based on 'Average Rank'
remaining_sorted = remaining_subset.sort_values('Average Rank')

# Assign ranks starting from 201 and ending at 1800
final.loc[remaining_sorted.index, '순위'] = range(201, 201 + len(remaining_sorted))

In [15]:
final


,Long Voting,Short Voting,Average Rank,Final Decision,순위
종목코드,,,,,
A000020,0,0,560.00,0,323
A000040,1,0,572.33,1,261
A000050,0,0,585.00,0,341
A000070,0,0,729.00,0,478
A000080,0,0,1437.67,0,1298
...,...,...,...,...,...
A375500,0,1,1149.33,1,2004
A378850,0,0,1681.33,0,1507
A383220,0,3,1942.67,1,1811


In [16]:
final = pd.DataFrame(final['순위'])



In [17]:
final.to_csv(subconfig.OUTPUT_PATH / 'Final Submission.csv')
